In [1]:
import os

from IPython.display import display, Image, Markdown
from getpass import getpass

import requests

In [2]:
%load_ext dotenv
%dotenv

In [3]:
if "CLOUDFLARE_API_TOKEN" in os.environ:
    api_token = os.environ["CLOUDFLARE_API_TOKEN"]
else:
    api_token = getpass("Enter you Cloudflare API Token")

In [4]:
if "CLOUDFLARE_ACCOUNT_ID" in os.environ:
    account_id = os.environ["CLOUDFLARE_ACCOUNT_ID"]
else:
    account_id = getpass("Enter your account id")

## Text Generation

In [5]:
model = "@cf/meta/llama-2-7b-chat-int8" 
#model = "@cf/mistral/mistral-7b-instruct-v0.1" 

response = requests.post(
    f"https://api.cloudflare.com/client/v4/accounts/{account_id}/ai/run/{model}",
    headers={"Authorization": f"Bearer {api_token}"},
    json={"messages": [
        {"role": "system", "content": "You are a productivity assistant for users of Jupyter notebooks for both Mac and Windows users. Respond in Markdown."},
        {"role": "user", "content": "How do I use keyboard shortcuts to execute cells?"}
    ]}
)

inference = response.json()
display(Markdown(inference["result"]["response"]))

Great question! 😊

### Mac Users:

To execute cells using keyboard shortcuts on a Mac, you can use the following combinations:

| Shortcut | Description |
| --- | --- |
| `Ctrl + Enter` | Execute the current cell. |
| `Shift + Enter` | Execute the current cell and move to the next cell. |
| `Cmd + Enter` | Execute the current cell and move to the next cell. (Only works in Jupyter Notebook 1.0 and later.) |

### Windows Users:

On Windows, you can use the following keyboard shortcuts to execute cells:

| Shortcut | Description |
| --- | --- |
| `Ctrl + Enter` | Execute the current cell. |
| `Shift + Enter` | Execute the current cell and move to the next cell. |
| `Shift + Ctrl + Enter` | Execute the current cell and move to the next cell without leaving the cell. |

Tips:

* You can also use the `F5` key to execute the current cell on both Mac and Windows.
*

## Text to Image

In [6]:
model = "@cf/stabilityai/stable-diffusion-xl-base-1.0"

response = requests.post(
    f"https://api.cloudflare.com/client/v4/accounts/{account_id}/ai/run/{model}",
    headers={"Authorization": f"Bearer {api_token}"},
    json={"prompt": "A pencil drawing of an excited developer using an API"}
)

display(Image(response.content))

## Translations

In [7]:
model = "@cf/meta/m2m100-1.2b"

response = requests.post(
    f"https://api.cloudflare.com/client/v4/accounts/{account_id}/ai/run/{model}",
    headers={"Authorization": f"Bearer {api_token}"},
    json={
        "text": "Artificial intelligence is pretty impressive these days. What do you think?",
        "source_lang": "english",
        "target_lang": "spanish"
    }
)

inference = response.json()
print(inference["result"]["translated_text"])

La inteligencia artificial es bastante impresionante en estos días. ¿Qué piensas?


## Text Classification

In [8]:
model = "@cf/huggingface/distilbert-sst-2-int8"

response = requests.post(
    f"https://api.cloudflare.com/client/v4/accounts/{account_id}/ai/run/{model}",
    headers={"Authorization": f"Bearer {api_token}"},
    json={"text": "This taco is delicious"}
)

inference = response.json()
inference["result"]

[{'label': 'NEGATIVE', 'score': 0.00012679687642958015},
 {'label': 'POSITIVE', 'score': 0.999873161315918}]

## Automatic Speech Recognition

In [9]:
model = "@cf/openai/whisper"

# TODO: Record me talking about speech to text
url = "https://ia904504.us.archive.org/6/items/Mitch_Hedburg_Thursday_ThrowTogether/mitch.mp3"
audio = requests.get(url)

response = requests.post(
    f"https://api.cloudflare.com/client/v4/accounts/{account_id}/ai/run/{model}",
    headers={"Authorization": f"Bearer {api_token}"},
    data=audio.content
)

inference = response.json()
inference

SSLError: HTTPSConnectionPool(host='ia904504.us.archive.org', port=443): Max retries exceeded with url: /6/items/Mitch_Hedburg_Thursday_ThrowTogether/mitch.mp3 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))

## Image Classification

In [ ]:
model = "@cf/microsoft/resnet-50"

# TODO: Record me talking about speech to text
url = "https://www.publicdomainpictures.net/pictures/140000/velka/enchilada-and-taco-meal.jpg"
image_request = requests.get(url)

display(Image(image_request.content, format="jpg"))
response = requests.post(
    f"https://api.cloudflare.com/client/v4/accounts/{account_id}/ai/run/{model}",
    headers={"Authorization": f"Bearer {api_token}"},
    data=image_request.content
)

inference = response.json()
inference